In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.datasets import make_classification

import sys, os
root = '/usr/local/serenceslab/maggie/shapeDim/'

sys.path.append(os.path.join(root, 'Analysis', 'code_utils'))
import data_utils



## Read in data and do x-session decoding

In [5]:
subjects = [1]#,2,3,4,5,7]

n_subj = len(subjects)

n_rois = 11

n_sess = 3

# penalties to eval
num_cs = 20
Cs = np.logspace( -5,0,num_cs )

# define model
model = LogisticRegressionCV( Cs=Cs, cv = 5, multi_class='multinomial', solver='lbfgs', penalty='l2', n_jobs = 8 )

# store acc...
acc = np.full( ( n_subj, n_rois, n_sess, n_sess, 6 ), np.nan )


task_names = ['Linear (1)','Linear (2)','Checker'];
n_tasks = len(task_names)

# three different ways to do binary decoding
n_bounds = 3;
bound_names = ['Decode: Linear (1)','Decode: Linear (2)','Decode: Checker'];
quad_groups = [[[1, 4], [2, 3]],
                [[1, 2], [3, 4]],
                [[1, 3], [2, 4]]];

make_time_resolved = False



# first load all data for all subjects, both tasks
maindat_all = []; repdat_all = []
mainlabs_all = []; replabs_all = []

for si, ss in enumerate(subjects):
    # si = 0; ss = 1;

    print('loading S%02d, main task'%ss)
    main_data, _, main_labels, roi_names = data_utils.load_main_task_data(ss, make_time_resolved)

    for ri in range(n_rois):
        
        # subtract mean across voxels each trial
        main_data[ri] -= np.tile(np.mean(main_data[ri], axis=1, keepdims=True), [1, main_data[ri].shape[1]])

    maindat_all += [main_data]
    mainlabs_all += [main_labels]

    print('loading S%02d, repeat task'%ss)
    rep_data, _, rep_labels, roi_names = data_utils.load_repeat_task_data(ss, make_time_resolved)

    for ri in range(n_rois):
        # subtract mean across voxels each trial
        rep_data[ri] -= np.tile(np.mean(rep_data[ri], axis=1, keepdims=True), [1, rep_data[ri].shape[1]])

    repdat_all += [rep_data]
    replabs_all += [rep_labels]
    
    ## make a new df that only has main grid trials
    grid_df = main_labels[ main_labels[ 'is_main_grid' ] == 1 ]

    # filter the data from each roi to only include main grid trials
    # move from list to dict so that roi name is the key (just my preference)
    grid_data = {}

    for r_idx, r in enumerate( roi_names ):

        grid_data[ r ] = main_data[ r_idx ][ main_labels[ 'is_main_grid' ] == 1 ] 


    # how many sessions
    n_sess = np.max( np.max( grid_df[ 'sess' ] ) )

    # how many parts in each session - check each separately...
    tmp = []
    for s in range( 1,n_sess+1 ):
        tmp.append( np.max( grid_df[ grid_df['sess'] == s ][ 'part' ] ) )
    
    n_parts = np.min( tmp ) 
    
    ## figure out grid position on each trial
    # get all x,y values
    x = np.array( grid_df[ 'ptx' ] )
    y = np.array( grid_df[ 'pty' ] )

    # meshgrid for all combos
    X,Y = np.meshgrid( np.unique(x),np.unique(y) )

    # figure out all unique x,y combos and put into array
    pnts = np.vstack( ( X.ravel(),Y.ravel() ) ).T

    # to store grid pos of each stim
    grid_pos = np.full( len( x ),np.nan )

    # loop over all trials
    for i in range( len( x ) ):

        # loop over unique points
        for j in range( len( pnts ) ):

            # find the match for current x,y pair
            if np.sum( [x[i],y[i]]==pnts[j,] ) == 2:

                # store
                grid_pos[i] = j

    # double check num unique vals + overall length
    assert( len( np.unique( grid_pos ) ) == 16 )
    assert( len( grid_pos ) == grid_data['V1'].shape[0] )    
    
    ## Model/predict...just do v1 at first...
    # loop over ROIs
    for r_idx, r in enumerate( roi_names ):

        # get the data from this ROI
        X = grid_data[ r ]

        # loop over sessions...train/test on all combos (e.g. sess1-1, sess1-2, sess1-3, etc)    
        for s in np.arange( 1,n_sess+1 ):

            # progress...
            print(f'ROI: {r}, Session: {s}')

            # loop over parts
            for p in np.arange( 1,n_parts+1 ):

                # training data from this session, all parts except current part...
                train_X = X[ ( grid_df['sess'] == s ) & ( grid_df['part'] != p ) ]
                train_y = grid_pos[ ( grid_df['sess'] == s ) & ( grid_df['part'] != p ) ]

                # train model...
                model.fit( train_X, train_y )

                # test data same session
                test_X = X[ ( grid_df['sess'] == s ) & ( grid_df['part'] == p ) ]
                test_y = grid_pos[ ( grid_df['sess'] == s ) & ( grid_df['part'] == p ) ]

                # predict
                acc[ si, r_idx, s-1, s-1, p-1 ] = model.score( test_X,test_y )

                # cross-gen from current session to other sessions...
                for cg_s in np.setdiff1d( range( 1,n_sess+1 ), s ):

                    # test data same session
                    test_X = X[ ( grid_df['sess'] == cg_s ) & ( grid_df['part'] == p ) ]
                    test_y = grid_pos[ ( grid_df['sess'] == cg_s ) & ( grid_df['part'] == p ) ]      

                    # predict
                    acc[ si, r_idx, s-1, cg_s-1, p-1 ] = model.score( test_X,test_y )
                    
                    

loading S01, main task
loading S01, repeat task
ROI: V1, Session: 1
ROI: V1, Session: 2
ROI: V1, Session: 3
ROI: V2, Session: 1
ROI: V2, Session: 2
ROI: V2, Session: 3
ROI: V3, Session: 1
ROI: V3, Session: 2
ROI: V3, Session: 3
ROI: V3AB, Session: 1
ROI: V3AB, Session: 2
ROI: V3AB, Session: 3


/home/AD/mmhender/anaconda3/envs/shapedim/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


ROI: hV4, Session: 1
ROI: hV4, Session: 2
ROI: hV4, Session: 3
ROI: IPS0, Session: 1
ROI: IPS0, Session: 2
ROI: IPS0, Session: 3
ROI: IPS1, Session: 1
ROI: IPS1, Session: 2
ROI: IPS1, Session: 3
ROI: IPS2, Session: 1
ROI: IPS2, Session: 2
ROI: IPS2, Session: 3
ROI: IPS3, Session: 1
ROI: IPS3, Session: 2
ROI: IPS3, Session: 3
ROI: LO1, Session: 1
ROI: LO1, Session: 2
ROI: LO1, Session: 3
ROI: LO2, Session: 1
ROI: LO2, Session: 2
ROI: LO2, Session: 3


In [8]:
np.mean(np.mean(np.mean(acc[0,:,:,:,:], axis=1), axis=1), axis=1)

array([0.16898148, 0.20572917, 0.17534722, 0.13136574, 0.0865162 ,
       0.07581019, 0.08015046, 0.06741898, 0.08043981, 0.11082176,
       0.08159722])

## Classify object as a function of task context...
* so train on task 1, generalize to task 2,3 in same session and in other sessions, etc...

In [ ]:
subjects = [1,2,3,4,5,7]

n_subj = len(subjects)

n_rois = 11

n_sess = 3

# task names...
task_names = ['Linear (1)','Linear (2)','Checker'];
n_tasks = len(task_names)

# three different ways to do binary decoding
n_bounds = 3;
bound_names = ['Decode: Linear (1)','Decode: Linear (2)','Decode: Checker'];
quad_groups = [[[1, 4], [2, 3]],
                [[1, 2], [3, 4]],
                [[1, 3], [2, 4]]];

# penalties to eval
num_cs = 20
Cs = np.logspace( -5,0,num_cs )

# define model
model = LogisticRegressionCV( Cs=Cs, cv = 5, multi_class='multinomial', solver='lbfgs', penalty='l2', n_jobs = 8 )

# store acc...
acc = np.full( ( n_subj, n_rois, n_tasks, n_tasks, n_sess ), np.nan )

make_time_resolved = False



# first load all data for all subjects, both tasks
maindat_all = []; repdat_all = []
mainlabs_all = []; replabs_all = []

for si, ss in enumerate(subjects):
    # si = 0; ss = 1;

    print('loading S%02d, main task'%ss)
    main_data, _, main_labels, roi_names = data_utils.load_main_task_data(ss, make_time_resolved)

    for ri in range(n_rois):
        
        # subtract mean across voxels each trial
        main_data[ri] -= np.tile(np.mean(main_data[ri], axis=1, keepdims=True), [1, main_data[ri].shape[1]])

    maindat_all += [main_data]
    mainlabs_all += [main_labels]

    print('loading S%02d, repeat task'%ss)
    rep_data, _, rep_labels, roi_names = data_utils.load_repeat_task_data(ss, make_time_resolved)

    for ri in range(n_rois):
        # subtract mean across voxels each trial
        rep_data[ri] -= np.tile(np.mean(rep_data[ri], axis=1, keepdims=True), [1, rep_data[ri].shape[1]])

    repdat_all += [rep_data]
    replabs_all += [rep_labels]
    
    ## make a new df that only has main grid trials
    grid_df = main_labels[ main_labels[ 'is_main_grid' ] == 1 ]

    # filter the data from each roi to only include main grid trials
    # move from list to dict so that roi name is the key (just my preference)
    grid_data = {}

    for r_idx, r in enumerate( roi_names ):

        grid_data[ r ] = main_data[ r_idx ][ main_labels[ 'is_main_grid' ] == 1 ] 


    # how many sessions
    n_sess = np.max( np.max( grid_df[ 'sess' ] ) )

    # how many parts in each session - check each separately...
    tmp = []
    for s in range( 1,n_sess+1 ):
        tmp.append( np.max( grid_df[ grid_df['sess'] == s ][ 'part' ] ) )
    
    n_parts = np.min( tmp ) 
    
    ## figure out grid position on each trial
    # get all x,y values
    x = np.array( grid_df[ 'ptx' ] )
    y = np.array( grid_df[ 'pty' ] )

    # meshgrid for all combos
    X,Y = np.meshgrid( np.unique(x),np.unique(y) )

    # figure out all unique x,y combos and put into array
    pnts = np.vstack( ( X.ravel(),Y.ravel() ) ).T

    # to store grid pos of each stim
    grid_pos = np.full( len( x ),np.nan )

    # loop over all trials
    for i in range( len( x ) ):

        # loop over unique points
        for j in range( len( pnts ) ):

            # find the match for current x,y pair
            if np.sum( [x[i],y[i]]==pnts[j,] ) == 2:

                # store
                grid_pos[i] = j

    # double check num unique vals + overall length
    assert( len( np.unique( grid_pos ) ) == 16 )
    assert( len( grid_pos ) == grid_data['V1'].shape[0] )    
    
    
    ## Model/predict...loop over ROIs
    for r_idx, r in enumerate( roi_names ):

        # get the data from this ROI
        X = grid_data[ r ]

        # loop over sessions...train/test on all combos (e.g. sess1-1, sess1-2, sess1-3, etc)    
        for s_idx,s in enumerate( np.arange( 1,n_sess+1 ) ):

            # progress...
            print(f'ROI: {r}, Session: {s}')

            # loop over task...
            for t_idx,t in enumerate( np.arange( 1,4 ) ):

                # training data from this session, task 1
                train_X = X[ ( grid_df['sess'] == s ) & ( grid_df['task'] == t ) ]
                train_y = grid_pos[ ( grid_df['sess'] == s ) & ( grid_df['task'] == t ) ]

                # train model...
                model.fit( train_X, train_y )

                # then generalize to other tasks...first same session, other tasks, then diff
                # sessions, other tasks. 
                for cgt_idx,cgt in enumerate( np.setdiff1d( range( 1,4 ), t ) ):

                    # test data same session
                    test_X = X[ ( grid_df['sess'] == s ) & ( grid_df['task'] == cgt ) ]
                    test_y = grid_pos[ ( grid_df['sess'] == s ) & ( grid_df['task'] == cgt ) ] 

                    acc[ si, r_idx, t_idx, cgt-1, s_idx ] = model.score( test_X,test_y )

#                     # cross-gen from current session to other sessions...
#                     for cgs_idx,cgs in enumerate( np.setdiff1d( range( 1,n_sess+1 ), s ) ):

#                         # test data same session
#                         test_X = X[ ( grid_df['sess'] == cgs ) & ( grid_df['task'] == t1[ trn_idx ] ) ]
#                         test_y = grid_pos[ ( grid_df['sess'] == cgs ) & ( grid_df['task'] == t1[ trn_idx ] ) ]      

#                         # predict
#                         acc[ si, r_idx, t_idx, cgt-1, s_idx, cgs-1 ] = model.score( test_X,test_y )
                    
                    

In [ ]:
na = np.nanmean(acc, axis = 0)

na[0,].shape


In [ ]:
plt.plot(np.sin(np.linspace(0,2*np.pi,1000)) * np.random.random(1000)*100)
plt.show()

In [ ]:
na = np.nanmean( na, axis = 2 )

In [ ]:
x = np.pi
y = 2/3 * np.pi
np.arctan2(y,x)

In [ ]:
roi = 7
print(roi_names[roi])

na = np.nanmean(acc, axis = 0)
na = np.nanmean( na, axis = 0)
#na = np.nanmean( na[roi,], axis = 0 )
na = np.nanmean( na, axis = 0 )
na = np.nanmean( na, axis = 0 )

plt.plot(na)
plt.show()

In [7]:
roi = 0
data = np.nanmean(na[roi,:2,:2,],axis = 0)
data = np.nanmean(data,axis = 0)

cgs = [[0,1],[0,2],[1,0],[1,2],[2,0],[2,1]]

# loop over cross-gen...
#for cg in cgs:
plt.imshow( data )
plt.colorbar()
plt.show()

plt.plot(data[:,0])
plt.plot(data[:,1])
plt.plot(data[:,2])
plt.legend(['Trn1','Trn2','Trn3'])
plt.show()

plt.plot(np.diag(data))
plt.show()

NameError: name 'na' is not defined

In [ ]:
np.diag( data )

In [ ]:
plt.imshow(na)
plt.colorbar()
plt.show()

plt.plot(na[0,:])
plt.plot(na[1,:])
plt.plot(na[2,:])
plt.show()

In [ ]:
for r_idx,r in enumerate( roi_names ):
  
    md = np.nanmean(acc[:,r_idx,:,:,:],axis=0)
    md = np.nanmean(md,axis=2)

    plt.imshow(md, cmap = 'RdBu',vmin = 0, vmax = .2)
    
    plt.title(r)
    
    plt.colorbar()
    plt.show()


## Read in data and do x-sess decoding of task

In [ ]:
grid_df['task'].unique()

In [ ]:
subjects = [1,2,3,4,5,7]

n_subj = len(subjects)

n_rois = 11

task_names = ['Linear (1)','Linear (2)','Checker'];
n_tasks = len(task_names)

# penalties to eval
num_cs = 20
Cs = np.logspace( -5,0,num_cs )

# define model
model = LogisticRegressionCV( Cs=Cs, cv = n_parts-1, multi_class='multinomial', solver='lbfgs', penalty='l2', n_jobs = 8 )

# store acc...
acc = np.full( ( n_subj, n_rois, n_sess, n_sess, 6 ), np.nan )

make_time_resolved = False

# first load all data for all subjects, both tasks
maindat_all = []; repdat_all = []
mainlabs_all = []; replabs_all = []

for si, ss in enumerate(subjects):
    # si = 0; ss = 1;

    print('loading S%02d, main task'%ss)
    main_data, _, main_labels, roi_names = data_utils.load_main_task_data(ss, make_time_resolved)

    for ri in range(n_rois):
        
        # subtract mean across voxels each trial
        main_data[ri] -= np.tile(np.mean(main_data[ri], axis=1, keepdims=True), [1, main_data[ri].shape[1]])

    maindat_all += [main_data]
    mainlabs_all += [main_labels]

    print('loading S%02d, repeat task'%ss)
    rep_data, _, rep_labels, roi_names = data_utils.load_repeat_task_data(ss, make_time_resolved)

    for ri in range(n_rois):
        # subtract mean across voxels each trial
        rep_data[ri] -= np.tile(np.mean(rep_data[ri], axis=1, keepdims=True), [1, rep_data[ri].shape[1]])

    repdat_all += [rep_data]
    replabs_all += [rep_labels]
    
    ## make a new df that only has main grid trials
    grid_df = main_labels[ main_labels[ 'is_main_grid' ] == 1 ]

    # filter the data from each roi to only include main grid trials
    # move from list to dict so that roi name is the key (just my preference)
    grid_data = {}

    for r_idx, r in enumerate( roi_names ):

        grid_data[ r ] = main_data[ r_idx ][ main_labels[ 'is_main_grid' ] == 1 ] 


    # how many sessions
    n_sess = np.max( np.max( grid_df[ 'sess' ] ) )

    # how many parts in each session - check each separately...
    tmp = []
    for s in range( 1,n_sess+1 ):
        tmp.append( np.max( grid_df[ grid_df['sess'] == s ][ 'part' ] ) )
    
    n_parts = np.min( tmp ) 
    
    ## figure out grid position on each trial
    # get all x,y values
    x = np.array( grid_df[ 'ptx' ] )
    y = np.array( grid_df[ 'pty' ] )

    # meshgrid for all combos
    X,Y = np.meshgrid( np.unique(x),np.unique(y) )

    # figure out all unique x,y combos and put into array
    pnts = np.vstack( ( X.ravel(),Y.ravel() ) ).T

    # to store grid pos of each stim
    grid_pos = np.full( len( x ),np.nan )

    # loop over all trials
    for i in range( len( x ) ):

        # loop over unique points
        for j in range( len( pnts ) ):

            # find the match for current x,y pair
            if np.sum( [x[i],y[i]]==pnts[j,] ) == 2:

                # store
                grid_pos[i] = j

    # double check num unique vals + overall length
    assert( len( np.unique( grid_pos ) ) == 16 )
    assert( len( grid_pos ) == grid_data['V1'].shape[0] )    
    
    ## Model/predict...just do v1 at first...so slow :(
    # loop over ROIs
    for r_idx, r in enumerate( roi_names ):

        # get the data from this ROI
        X = grid_data[ r ]
        y = grid_df['task']
        
        # loop over sessions...train/test on all combos (e.g. sess1-1, sess1-2, sess1-3, etc)    
        for s in np.arange( 1,n_sess+1 ):

            # progress...
            print(f'ROI: {r}, Session: {s}')

            # loop over parts
            for p in range(1,3):

                if p==1:

                    # training data from this session, all parts except current part...
                    train_X = X[ ( grid_df['sess'] == s ) & ( grid_df['part'] < 4 ) ]
                    train_y = y[ ( grid_df['sess'] == s ) & ( grid_df['part'] < 4 ) ]

                    # train model...
                    model.fit( train_X, train_y )

                    # test data same session
                    test_X = X[ ( grid_df['sess'] == s ) & ( grid_df['part'] > 3 ) ]
                    test_y = y[ ( grid_df['sess'] == s ) & ( grid_df['part'] > 3 ) ]

                    # predict
                    acc[ si, r_idx, s-1, s-1, p-1 ] = model.score( test_X,test_y )

                    # cross-gen from current session to other sessions...
                    for cg_s in np.setdiff1d( range( 1,n_sess+1 ), s ):

                        # test data same session
                        test_X = X[ ( grid_df['sess'] == cg_s ) & ( grid_df['part'] > 3 ) ]
                        test_y = y[ ( grid_df['sess'] == cg_s ) & ( grid_df['part'] > 3 ) ]      

                        # predict
                        acc[ si, r_idx, s-1, cg_s-1, p-1 ] = model.score( test_X,test_y )
                    
                else:
                
                    # training data from this session, all parts except current part...
                    train_X = X[ ( grid_df['sess'] == s ) & ( grid_df['part'] > 3 ) ]
                    train_y = y[ ( grid_df['sess'] == s ) & ( grid_df['part'] > 3 ) ]

                    # train model...
                    model.fit( train_X, train_y )

                    # test data same session
                    test_X = X[ ( grid_df['sess'] == s ) & ( grid_df['part'] < 4 ) ]
                    test_y = y[ ( grid_df['sess'] == s ) & ( grid_df['part'] < 4 ) ]

                    # predict
                    acc[ si, r_idx, s-1, s-1, p-1 ] = model.score( test_X,test_y )

                    # cross-gen from current session to other sessions...
                    for cg_s in np.setdiff1d( range( 1,n_sess+1 ), s ):

                        # test data same session
                        test_X = X[ ( grid_df['sess'] == cg_s ) & ( grid_df['part'] < 4 ) ]
                        test_y = y[ ( grid_df['sess'] == cg_s ) & ( grid_df['part'] < 4 ) ]      

                        # predict
                        acc[ si, r_idx, s-1, cg_s-1, p-1 ] = model.score( test_X,test_y )

In [ ]:
acc[:,0,:,:,1]

In [ ]:
plt.plot(np.array([grid_df['sess'],grid_df['part'],grid_df['run_in_part'],grid_df['task']]).T)
plt.show()

In [ ]:
for r_idx,r in enumerate( roi_names ):
  
    md = np.nanmean(acc[:,r_idx,:,:,:2],axis=0)
    md = np.nanmean(md,axis=2)

    plt.imshow(md, cmap = 'RdBu',vmin = .33, vmax = .4)
    
    plt.title(r)
    
    plt.colorbar()
    plt.show()

## First filter the `is_main_grid` trials out and grab a few constants we'll use for parsing the data during decoding...

In [ ]:
# make a new df that only has main grid trials
grid_df = main_labels[ main_labels[ 'is_main_grid' ] == 1 ]

# filter the data from each roi to only include main grid trials
# move from list to dict so that roi name is the key (just my preference)
grid_data = {}

for r_idx, r in enumerate( roi_names ):
    
    grid_data[ r ] = main_data[ r_idx ][ main_labels[ 'is_main_grid' ] == 1 ] 
    
    
# how many sessions
n_sess = np.max( np.max( grid_df[ 'sess' ] ) )

# how many parts in each session 
n_parts = np.max( grid_df[ 'part' ] ) 


## now assign label to each of the 16 positions...this is clunky, but whatever...

In [ ]:
# get all x,y values
x = np.array( grid_df[ 'ptx' ] )
y = np.array( grid_df[ 'pty' ] )

# meshgrid for all combos
X,Y = np.meshgrid( np.unique(x),np.unique(y) )

# figure out all unique x,y combos and put into array
pnts = np.vstack( ( X.ravel(),Y.ravel() ) ).T

# to store grid pos of each stim
grid_pos = np.full( len( x ),np.nan )

# loop over all trials
for i in range( len( x ) ):
    
    # loop over unique points
    for j in range( len( pnts ) ):
        
        # find the match for current x,y pair
        if np.sum( [x[i],y[i]]==pnts[j,] ) == 2:
            
            # store
            grid_pos[i] = j
            
# double check num unique vals + overall length
assert( len( np.unique( grid_pos ) ) == 16 )
assert( len( grid_pos ) == grid_data['V1'].shape[0] )

## Do decoding using multinomial logistic regression - here doing within and across-session generalization using LOO (leave one part out in this case)
* note: `roi_names == ['V1', 'V2', 'V3', 'V3AB', 'hV4', 'IPS0', 'IPS1', 'IPS2', 'IPS3', 'LO1', 'LO2']`

In [ ]:
# penalties to eval
num_cs = 10
Cs = np.logspace( -5,0,num_cs )

# define model
# model = LogisticRegressionCV( Cs=Cs, cv = n_parts-1, multi_class='multinomial', solver='lbfgs', penalty='l2' )
model = LogisticRegressionCV( cv = n_parts-1, multi_class='multinomial', solver='lbfgs', penalty='l2', n_jobs=-1 )

# store acc...
acc = np.full( ( len( roi_names ), n_sess, n_sess, n_parts ), np.nan )

# loop over ROIs
for r_idx, r in enumerate( roi_names[:2] ):

    # get the data from this ROI
    X = grid_data[ r ]

    # loop over sessions...train/test on all combos (e.g. sess1-1, sess1-2, sess1-3, etc)    
    for s in np.arange( 1,n_sess+1 ):

        # progress...
        print(f'ROI: {r}, Session: {s}')
        
        # loop over parts
        for p in np.arange( 1,n_parts+1 ):
              
            # training data from this session, all parts except current part...
            train_X = X[ ( grid_df['sess'] == s ) & ( grid_df['part'] != p ) ]
            train_y = grid_pos[ ( grid_df['sess'] == s ) & ( grid_df['part'] != p ) ]

            # train model...
            model.fit( train_X, train_y )
            
            # test data same session
            test_X = X[ ( grid_df['sess'] == s ) & ( grid_df['part'] == p ) ]
            test_y = grid_pos[ ( grid_df['sess'] == s ) & ( grid_df['part'] == p ) ]

            # predict
            acc[ r_idx, s-1, s-1, p-1 ] = model.score( test_X,test_y )
            
            # cross-gen from current session to other sessions...
            for cg_s in np.setdiff1d( range( 1,n_sess+1 ), s ):

                # test data same session
                test_X = X[ ( grid_df['sess'] == cg_s ) & ( grid_df['part'] == p ) ]
                test_y = grid_pos[ ( grid_df['sess'] == cg_s ) & ( grid_df['part'] == p ) ]      
                
                # predict
                acc[ r_idx, s-1, cg_s-1, p-1 ] = model.score( test_X,test_y )
    
    
    

In [ ]:
na = np.mean(acc[0,:,:,:], axis=2)

In [ ]:
plt.imshow( na )
plt.colorbar()
plt.show()

In [ ]:
X, y = make_classification(n_samples= 1000, n_features= 9,
                           n_classes=2,random_state=42)

train_X = X[:900,:]
train_y = y[:900]
test_X = X[900:,:]
test_y = y[900:]

model = LogisticRegressionCV( cv = 10, multi_class='multinomial', solver='lbfgs', penalty='l2' )

model.fit(train_X,train_y)
cv_acc = model.score(test_X,test_y)




## Quick plot of classification acc as a function of penalty

In [ ]:
plt.plot( pens, acc, linewidth = 2 )
plt.hlines( 1/16, pens[0], pens[-1] )
plt.xscale( 'log' )
plt.ylim( [ 0, 0.5 ] )
plt.xlabel( 'Penalty' )
plt.ylabel( 'Accuracy' )
plt.show()
